## Importing Libraries

In [28]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv
from datetime import datetime
from sqlalchemy import types, create_engine
from ulid import ULID
import warnings
warnings.filterwarnings("ignore")

## Load env file

In [29]:
## load env
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
load_dotenv(os.path.join(BASE_DIR, '.env'))

True

## Retrieving data from Sitemap

In [30]:
base_url = os.getenv("BASE_URL")
request_url = f"{base_url}/GhanaHomePage/sitemap.php"
page = requests.get(request_url)
# print(page.text)

In [31]:
soup = BeautifulSoup(page.content, 'html.parser')

In [32]:
results = soup.find(id = "medsection2")

In [33]:
# print(results.prettify())

In [34]:
site_map_data = results.find_all("dl", class_ = "resources")

In [35]:
df_columns = ['section','url']
sections_df = pd.DataFrame(columns = df_columns)

In [36]:
for sections in site_map_data:
    urls = sections.find_all("a")
    for url in urls:
        temp_row = pd.DataFrame(data = [[url.text,base_url+url.get('href')]], columns = df_columns)
        sections_df = pd.concat([sections_df,temp_row], ignore_index=True)

## Selecting Story Sections with articles

In [37]:
section_list = ["Abroad, Ghanaians","AFCON","African News","Athletics",
                "BBC Hausa News", "BBC Pidgin News","Business & Economy",
                "Coronavirus","Crime News","Editorial News","Entertainment",
                "Health News","Lifestyle","Music","Political News",
                "Regional News","Sports Section","Tabloid News"]

In [38]:
# getting the urls to the different sectiions
def get_section_urls(row):
    df = pd.DataFrame()
    if row['section'] in section_list:
        temp_row = pd.DataFrame(data = [[row["section"], row["url"]]], columns = df_columns)
        df = pd.concat([df,temp_row],ignore_index =True)
    else:
        pass
    return df

In [39]:
df = sections_df.apply(lambda row : get_section_urls(row),axis = 1)

In [40]:
df_columns = ['section','url']
stories_urls_df = pd.DataFrame(columns = df_columns)

In [41]:
for row in df:
    stories_urls_df = pd.concat([stories_urls_df,row])

In [42]:
stories_urls_df.reset_index(inplace=True,drop=True)

## Generate ULIDs for the various sections

In [43]:
## function to generate ulid
def generate_ulid(row):
    row["id"] = int(ULID())
    return row

In [44]:
stories_urls_df = stories_urls_df.apply(lambda row: generate_ulid(row), axis=1)

In [45]:
stories_urls_df["dateGenerated"] = datetime.now()

In [46]:
stories_urls_df = stories_urls_df[["id","section","url","dateGenerated"]]

## Push data to db

In [48]:
# create engine
MYSQL_DB_USER = os.getenv("MYSQL_DB_USER")
MYSQL_DB_PASSWORD = os.getenv("MYSQL_DB_PASSWORD")
MYSQL_DB_HOST = os.getenv("MYSQL_DB_HOST")
MYSQL_DB_PORT = os.getenv("MYSQL_DB_PORT")
MYSQL_DB_NAME = os.getenv("MYSQL_DB_NAME") 

mysql_engine = create_engine(f"mysql+pymysql://{MYSQL_DB_USER}:{MYSQL_DB_PASSWORD}@{MYSQL_DB_HOST}:{MYSQL_DB_PORT}/{MYSQL_DB_NAME}")

In [53]:
## create df schema
df_schema = dict(zip(stories_urls_df.columns.tolist(),(types.VARCHAR(length=100), 
                                                    types.VARCHAR(length=150),
                                                    types.VARCHAR(length=1000),
                                                    types.TIMESTAMP)))

In [54]:
stories_urls_df.to_sql("sections", con=mysql_engine, if_exists="replace",dtype=df_schema,index=False) 

18